In [ ]:
import requests
import json
import time
import os
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

# API Key securely loaded from .env
API_KEY = os.getenv('GEMINI_API_KEY')

if not API_KEY:
    print("Warning: GEMINI_API_KEY not found in environment variables.")

def test_models_raw_http():
    print("Versuche direkten HTTP-Zugriff auf die API...")
    
    list_url = f"https://generativelanguage.googleapis.com/v1beta/models?key={API_KEY}"
    
    results = []
    
    try:
        response = requests.get(list_url, timeout=10)
        
        if response.status_code != 200:
            print(f"Fehler beim Abrufen der Liste: {response.status_code} - {response.text}")
            return
        
        data = response.json()
        models = data.get('models', [])
        print(f"{len(models)} Modelle gefunden. Starte Tests...\n")
        
        for model in models:
            model_name = model['name']
            methods = model.get('supportedGenerationMethods', [])
            
            if "generateContent" in methods:
                print(f"--- Teste: {model_name} ---")
                
                generate_url = f"https://generativelanguage.googleapis.com/v1beta/{model_name}:generateContent?key={API_KEY}"
                
                payload = {
                    "contents": [{
                        "parts": [{"text": "Antworte nur mit einem Wort: Hallo."}]
                    }]
                }
                
                status_msg = ""
                is_success = False
                
                try:
                    gen_response = requests.post(
                        generate_url, 
                        headers={"Content-Type": "application/json"},
                        data=json.dumps(payload),
                        timeout=10
                    )
                    
                    if gen_response.status_code == 200:
                        status_msg = "OK"
                        is_success = True
                        print("Status: OK")
                        try:
                            ans = gen_response.json()['candidates'][0]['content']['parts'][0]['text']
                            print(f"Antwort: {ans.strip()}")
                        except Exception:
                           pass
                    elif gen_response.status_code == 429:
                        status_msg = "Rate Limit (429)"
                        print("Status: Rate Limit (429)")
                        time.sleep(5)
                    else:
                        status_msg = f"Fehler {gen_response.status_code}"
                        print(status_msg)
                        
                except Exception as e:
                    status_msg = f"Fehler: {str(e)}"
                    print(status_msg)
                
                results.append((is_success, model_name, status_msg))
                print("")
                time.sleep(1)

    except Exception as e:
        print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

    # Sort results: Success (True) first
    results.sort(key=lambda x: x[0], reverse=True)
    
    # Calculate stats
    total = len(results)
    success_count = sum(1 for r in results if r[0])
    
    # Generate HTML Report
    date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    filename = "gemini_rate_check_results.html"
    
    # NOTE: Double curly braces {{ }} needed for CSS in f-strings!
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <meta charset='utf-8'>
    <title>Gemini Rate Check Results</title>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; background: #09090b; color: #e4e4e7; padding: 40px; max-width: 800px; margin: 0 auto; }}
        h1 {{ text-align: center; color: #fff; margin-bottom: 5px; }}
        .meta {{ text-align: center; color: #71717a; font-size: 0.9em; margin-bottom: 30px; letter-spacing: 0.5px; }}
        .stats {{ display: flex; justify-content: center; gap: 20px; margin-bottom: 30px; }}
        .stat-box {{ background: #18181b; padding: 15px 25px; border-radius: 12px; border: 1px solid #27272a; text-align: center; }}
        .stat-value {{ font-size: 24px; font-weight: bold; color: #fff; }}
        .stat-label {{ font-size: 12px; color: #a1a1aa; text-transform: uppercase; letter-spacing: 1px; }}
        
        table {{ border-collapse: separate; border-spacing: 0; width: 100%; border: 1px solid #27272a; border-radius: 12px; overflow: hidden; background: #18181b; }}
        th, td {{ padding: 16px 20px; text-align: left; border-bottom: 1px solid #27272a; }}
        th {{ background: #27272a; color: #fff; font-weight: 600; font-size: 14px; }}
        tr:last-child td {{ border-bottom: none; }}
        tr:hover td {{ background: #27272a; }}
        
        .status-badge {{ display: inline-flex; items-center; padding: 4px 10px; border-radius: 20px; font-size: 12px; font-weight: 600; }}
        .status-success {{ background: rgba(34, 197, 94, 0.1); color: #4ade80; border: 1px solid rgba(34, 197, 94, 0.2); }}
        .status-fail {{ background: rgba(239, 68, 68, 0.1); color: #f87171; border: 1px solid rgba(239, 68, 68, 0.2); }}
        
        .model-name {{ font-family: 'Monaco', 'Consolas', monospace; color: #e4e4e7; }}
    </style>
    </head>
    <body>
        <h1>Gemini Model Status</h1>
        <div class="meta">Generated on {date_str}</div>
        
        <div class="stats">
            <div class="stat-box">
                <div class="stat-value" style="color: #3b82f6">{total}</div>
                <div class="stat-label">Total Models</div>
            </div>
            <div class="stat-box">
                <div class="stat-value" style="color: #4ade80">{success_count}</div>
                <div class="stat-label">Operational</div>
            </div>
        </div>

        <table>
            <thead>
                <tr>
                    <th width="100">Status</th>
                    <th>Model Name</th>
                    <th>Message</th>
                </tr>
            </thead>
            <tbody>
    """
    
    for success, name, msg in results:
        status_class = "status-success" if success else "status-fail"
        status_text = "Operational" if success else "Failed"
        html_content += f"""
                <tr>
                    <td><span class="status-badge {status_class}">{status_text}</span></td>
                    <td class="model-name">{name}</td>
                    <td>{msg}</td>
                </tr>
        """
        
    html_content += """
            </tbody>
        </table>
    </body>
    </html>
    """
    
    # Check if we are in a read-only filesystem (sometimes an issue in sandboxes)
    # But here we assume we can write.
    try:
        with open(filename, "w") as f:
            f.write(html_content)
        print(f"\nReport generated: {filename}")
    except Exception as e:
        print(f"Error writing file: {e}")

if __name__ == "__main__":
    test_models_raw_http()